In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta

from scipy.stats import norm
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV

from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

import statsmodels.formula.api as smf
import statsmodels.api as sm

import pickle

## Step 1: Read in hold out data, scalers, and best model

In [2]:
hold_out = pd.read_csv('kc_house_data_test_features.csv')
kc_df = hold_out
# zip_score = pd.read_csv('Niche.csv')[['zipcode']]
# kc_df = zip_score.set_index('zipcode').join(kc_df.set_index('zipcode'))

In [3]:
# scaler_file = open('scaler.pickle','rb')
# final_scaler = pickle.load(scaler_file)
model_file = open('model.pickle','rb')
final_model = pickle.load(model_file)
# scaler_file.close()
model_file.close()

In [4]:
final_model.coef_

array([-1.79016469e+04,  2.55676008e+04,  1.34322982e+02,  7.27180718e-01,
       -3.14526247e+04,  5.07290667e+04,  3.20740858e+04,  3.98061637e+01,
       -1.53775320e+04, -1.15485605e+03,  1.42874942e+01, -4.11785813e-03,
        1.62331867e+04,  7.74142816e+05,  2.91456347e+05,  2.42538655e+05,
        2.49700382e+05,  2.55943780e+05,  5.20375919e+04,  1.34408830e+05,
        8.58145214e+04,  8.47232891e+04, -3.14684463e+04, -3.52853342e+04,
        1.60025722e+05,  1.62798250e+05,  1.31331280e+05,  2.13854714e+05,
        3.50382064e+05,  1.99009383e+05,  2.74929905e+04,  1.20284950e+06,
        4.86222574e+05,  9.30777099e+04,  2.35370825e+05,  2.03064786e+05,
        3.96795427e+04,  8.47151548e+04,  3.19688853e+04,  7.21536174e+04,
        8.44967213e+04,  1.54561110e+05,  1.74157699e+05,  1.58985735e+05,
        1.01563196e+05, -3.43519182e+04,  5.02080551e+05,  3.53144569e+05,
        4.85565992e+05,  1.28437637e+05,  3.57892070e+05,  1.24139598e+05,
        5.09323033e+05,  

## Step 2: Feature Engineering for holdout set

In [5]:
#Keeping the ID column
kc_id = kc_df['id']
#dropping the ID
kc_df.drop(['id', 'Unnamed: 0', 'date'], axis = 1, inplace = True)

In [48]:
# kc_df = kc_df.reset_index()

In [49]:
kc_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [6]:
def n_bath(row):
    if row['bathrooms'] == 0:
        row['bathrooms'] = 1
    if row['bathrooms'] > 10:
        row['bathrooms'] = 10
    return row
kc_df = kc_df.apply(n_bath, axis = 1)

In [7]:
def n_bed(row):
    if row['bedrooms'] ==0:
        row['bedrooms'] = row['floors']
    if row['bedrooms'] > 10:
        row['bedrooms'] = 10
    return row
kc_df = kc_df.apply(n_bed, axis = 1)

In [8]:
kc_df['sqft_basement'] = np.where(kc_df['sqft_basement'] != 0, 1, 0)
#Sets sqft_basement to categorical data
kc_df['master_br'] = np.where(kc_df['bedrooms'] > 2, 1, 0)
#Sets master_br variable being likely in a house

In [53]:
# def city_distance(row):
#     dist = geopy.distance.geodesic((47.6205,122.3493), (row['lat'], abs(row['long'])))
#     return dist.miles

In [54]:
# kc_df['dist_city'] = kc_df.apply(city_distance, axis=1)

In [9]:
kc_df.drop(['lat', 'long'], axis = 1, inplace = True)

In [10]:
cat_var = ['zipcode', 'grade', 'waterfront']
cont_var = ['bedrooms', 'bathrooms', 'dist_city', 'sqft_living', 'sqft_lot', 'yr_updated']

In [11]:
def to_dummies(df, features):
    """
    Generating dummy variables for feature
    parama: df
            features - list of features
    """
    d_df = []
    for f in features:
        d_df.append(pd.get_dummies(df[f], prefix='{}'.format(str(f)[:3]), drop_first=True))
    df = df.drop(features, axis = 1)
    df = pd.concat([df] + d_df ,axis=1)
    return df

In [12]:
kc_df = to_dummies(kc_df, cat_var)


In [13]:
kc_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,sqft_above,sqft_basement,yr_built,...,gra_5.0,gra_6.0,gra_7.0,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0
0,4.0,2.50,2270.0,11500.0,1.0,0.0,3.0,1540.0,1,1967.0,...,0,0,0,1,0,0,0,0,0,0
1,4.0,2.50,2270.0,11500.0,1.0,0.0,3.0,1540.0,1,1967.0,...,0,0,0,1,0,0,0,0,0,0
2,3.0,2.50,1470.0,1779.0,2.0,0.0,3.0,1160.0,1,2005.0,...,0,0,0,1,0,0,0,0,0,0
3,3.0,1.75,1280.0,16200.0,1.0,0.0,3.0,1030.0,1,1976.0,...,0,0,0,1,0,0,0,0,0,0
4,4.0,2.75,2830.0,8126.0,2.0,0.0,3.0,2830.0,0,2005.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3.0,2.50,1530.0,1131.0,3.0,0.0,3.0,1530.0,0,2009.0,...,0,0,0,1,0,0,0,0,0,0
4319,4.0,2.50,2310.0,5813.0,2.0,0.0,3.0,2310.0,0,2014.0,...,0,0,0,1,0,0,0,0,0,0
4320,2.0,0.75,1020.0,1350.0,2.0,0.0,3.0,1020.0,0,2009.0,...,0,0,1,0,0,0,0,0,0,0
4321,3.0,2.50,1600.0,2388.0,2.0,0.0,3.0,1600.0,0,2004.0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
def yr_update(row):
    if row['yr_renovated'] == 0:
        row['yr_updated'] = 2021 - row['yr_built']
    elif row['yr_renovated'] != 0:
        row['yr_updated'] = 2021 - row['yr_renovated']
    return row['yr_updated']

In [15]:
kc_df['yr_updated'] = kc_df.apply(yr_update, axis=1)

In [62]:
# kc_df.drop(columns = ['yr_built', 'yr_renovated'], axis = 1, inplace = True)

In [16]:
kc_df['sqft_living_sqft_lot'] = kc_df['sqft_living'] * kc_df['sqft_lot']

In [17]:
kc_df['grad_3.0'] = 0

In [65]:
kc_df['grad_3.0']

0       0
1       0
2       0
3       0
4       0
       ..
4318    0
4319    0
4320    0
4321    0
4322    0
Name: grad_3.0, Length: 4323, dtype: int64

In [66]:
kc_df

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15,master_br,dist_city,...,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0,yr_updated,sqft_living_sqft_lot,grad_3.0
0,4.0,2.50,2270.0,11500.0,1540.0,1,2020.0,10918.0,1,7.927458,...,1,0,0,0,0,0,0,54.0,26105000.0,0
1,4.0,2.50,2270.0,11500.0,1540.0,1,2020.0,10918.0,1,7.927458,...,1,0,0,0,0,0,0,54.0,26105000.0,0
2,3.0,2.50,1470.0,1779.0,1160.0,1,1470.0,1576.0,1,17.183470,...,1,0,0,0,0,0,0,16.0,2615130.0,0
3,3.0,1.75,1280.0,16200.0,1030.0,1,1160.0,10565.0,1,15.487328,...,1,0,0,0,0,0,0,45.0,20736000.0,0
4,4.0,2.75,2830.0,8126.0,2830.0,0,2830.0,7916.0,1,13.482364,...,1,0,0,0,0,0,0,16.0,22996580.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3.0,2.50,1530.0,1131.0,1530.0,0,1530.0,1509.0,1,5.446183,...,1,0,0,0,0,0,0,12.0,1730430.0,0
4319,4.0,2.50,2310.0,5813.0,2310.0,0,1830.0,7200.0,1,7.608764,...,1,0,0,0,0,0,0,7.0,13428030.0,0
4320,2.0,0.75,1020.0,1350.0,1020.0,0,1020.0,2007.0,0,2.962102,...,0,0,0,0,0,0,0,12.0,1377000.0,0
4321,3.0,2.50,1600.0,2388.0,1600.0,0,1410.0,1287.0,1,14.387336,...,1,0,0,0,0,0,0,17.0,3820800.0,0


Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [28]:
transformed_holdout = final_scaler.transform(kc_df)

In [207]:
# kc_df.drop('index', axis = 1, inplace = True)

In [19]:
best_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'sqft_living15',
       'sqft_lot15', 'master_br', 'zip_98004.0', 'zip_98005.0', 'zip_98006.0',
       'zip_98007.0', 'zip_98008.0', 'zip_98010.0', 'zip_98011.0',
       'zip_98014.0', 'zip_98019.0', 'zip_98022.0', 'zip_98023.0',
       'zip_98024.0', 'zip_98027.0', 'zip_98028.0', 'zip_98029.0',
       'zip_98033.0', 'zip_98034.0', 'zip_98038.0', 'zip_98039.0',
       'zip_98040.0', 'zip_98045.0', 'zip_98052.0', 'zip_98053.0',
       'zip_98055.0', 'zip_98056.0', 'zip_98058.0', 'zip_98059.0',
       'zip_98065.0', 'zip_98072.0', 'zip_98074.0', 'zip_98075.0',
       'zip_98077.0', 'zip_98092.0', 'zip_98102.0', 'zip_98103.0',
       'zip_98105.0', 'zip_98106.0', 'zip_98107.0', 'zip_98108.0',
       'zip_98109.0', 'zip_98112.0', 'zip_98115.0', 'zip_98116.0',
       'zip_98117.0', 'zip_98118.0', 'zip_98119.0', 'zip_98122.0',
       'zip_98125.0', 'zip_98126.0', 'zip_98133.0', 'zip_98136.0',
       'zip_98144.0', 'zip_98146.0', 'zip_98148.0', 'zip_98155.0',
       'zip_98166.0', 'zip_98168.0', 'zip_98177.0', 'zip_98178.0',
       'zip_98199.0', 'gra_5.0', 'gra_6.0', 'gra_7.0', 'gra_8.0', 'gra_9.0',
       'gra_10.0', 'gra_11.0', 'gra_12.0', 'gra_13.0', 'wat_1.0', 'yr_updated',
       'sqft_living_sqft_lot']

In [20]:
len(best_features)

85

## Step 3: Predict the holdout set

In [21]:
final_answers = final_model.predict(kc_df[best_features])

In [22]:
final_answers[:10]

array([526553.35989142, 526553.35989142, 386799.36412885, 264798.57562696,
       518064.89120282, 593493.66711429, 302389.7325296 , 289012.78808849,
       402795.57072597, 274344.30429871])

In [24]:
df=pd.DataFrame(final_answers)

In [25]:
df

,0
0,526553.359891
1,526553.359891
2,386799.364129
3,264798.575627
4,518064.891203
...,...
4318,532494.138436
4319,431459.390622
4320,343073.242221
4321,385660.481862


## Step 4: Export your predictions

In [27]:
df.to_csv('housing_preds_christa.csv')

In [18]:
kc_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,sqft_above,sqft_basement,yr_built,...,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0,yr_updated,sqft_living_sqft_lot,grad_3.0
0,4.0,2.50,2270.0,11500.0,1.0,0.0,3.0,1540.0,1,1967.0,...,1,0,0,0,0,0,0,54.0,26105000.0,0
1,4.0,2.50,2270.0,11500.0,1.0,0.0,3.0,1540.0,1,1967.0,...,1,0,0,0,0,0,0,54.0,26105000.0,0
2,3.0,2.50,1470.0,1779.0,2.0,0.0,3.0,1160.0,1,2005.0,...,1,0,0,0,0,0,0,16.0,2615130.0,0
3,3.0,1.75,1280.0,16200.0,1.0,0.0,3.0,1030.0,1,1976.0,...,1,0,0,0,0,0,0,45.0,20736000.0,0
4,4.0,2.75,2830.0,8126.0,2.0,0.0,3.0,2830.0,0,2005.0,...,1,0,0,0,0,0,0,16.0,22996580.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3.0,2.50,1530.0,1131.0,3.0,0.0,3.0,1530.0,0,2009.0,...,1,0,0,0,0,0,0,12.0,1730430.0,0
4319,4.0,2.50,2310.0,5813.0,2.0,0.0,3.0,2310.0,0,2014.0,...,1,0,0,0,0,0,0,7.0,13428030.0,0
4320,2.0,0.75,1020.0,1350.0,2.0,0.0,3.0,1020.0,0,2009.0,...,0,0,0,0,0,0,0,12.0,1377000.0,0
4321,3.0,2.50,1600.0,2388.0,2.0,0.0,3.0,1600.0,0,2004.0,...,1,0,0,0,0,0,0,17.0,3820800.0,0
